<a href="https://colab.research.google.com/github/cavrinceanu/earthenginescripts/blob/master/ERA5_S1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Download matching ERA-5 Hourly Data on single levels from 1979 to present for Sentinel-1 imagery

Created by: 
Cristina Vrinceanu, 
University of Nottingham, 
cristina.vrinceanu@nottingham.ac.uk

This notebook provides the code to couple Sentinel-1 data with ERA-5 daily hourly data from the reanalysis dataset. This has various applications in the marine domain. 


The code matches Sentinel-1 scene metadata to ERA-5 parameters and downloads 10-m u and v wind vector components in netcdf format for the following parameters: 



*   Area of interest (scene footprint)
*   Hour of acquisition (matching the closest time)
*   Date of acquisition (Year, Month, Day)



Documentation:   

*   [ERA5 hourly data on single levels from 1979 to present](https://cds.climate.copernicus.eu/cdsapp#!/dataset/reanalysis-era5-single-levels?tab=overview)
*   [Climate Data Store API](https://cds.climate.copernicus.eu/api-how-to)
*   [Sentinel-1 Documentation in GEE](https://developers.google.com/earth-engine/datasets/catalog/COPERNICUS_S1_GRD)
*   [Sentinel-1 Mission Documentation ](https://sentinels.copernicus.eu/web/sentinel/missions/sentinel-1)
*   [Google Earth Engine Documentation](https://developers.google.com/earth-engine/apidocs/export-table-toasset) 





Requirements: Gmail account, Google Earth Engine account, Copernicus Climate Services CDS account, free storage space in Google Drive.

## Initialise Google Earth Engine and necessary libraries

In [ ]:
import ee        #GEE library 
import pandas as pd
import numpy as np
import timeit

In [ ]:
ee.Authenticate()   #Colab requires authentification to Google. Grant permission and copy/paste the code in the box and hit enter.
ee.Initialize()

Mount personal Google Drive. You can do this operation in the side panel or run the following cells.


In [ ]:
#from google.colab import drive
#drive.mount('content/drive/')

## Access the Sentinel-1 product collection


Access the Sentinel-1 data from Google Earth Engine and extract the time of acquisition metadata.

In [ ]:
#define dates
start_date = ee.Date('2018-08-20')
end_date = ee.Date('2018-08-31')

In [ ]:
#define geometrtry from coordinates 
lat, lng = 41.14, 40.68
aoi = ee.Geometry.Point(lat, lng)

In [ ]:
#filter Image Collection
s1=ee.ImageCollection("COPERNICUS/S1_GRD").filterBounds(aoi).filterDate(start_date, end_date)

In [ ]:
#Extract number of Sentinel-1 products
count = s1.size()
col_count=count.getInfo()
print('Count: ', str(col_count)+'\n')

## Extract metadata and footprints 

In this step, we use the geemap library for a quick and interactive visualization. 

*   Install geemap: https://pypi.org/project/geemap/
*   geemap documentation: https://geemap.org/


    Wu, Q., (2020). geemap: A Python package for interactive mapping with Google Earth Engine. The Journal of Open Source Software, 5(51), 2305. https://doi.org/10.21105/joss.02305
    Wu, Q., Lane, C. R., Li, X., Zhao, K., Zhou, Y., Clinton, N., DeVries, B., Golden, H. E., & Lang, M. W. (2019). Integrating LiDAR data and multi-temporal aerial imagery to map wetland inundation dynamics using Google Earth Engine. Remote Sensing of Environment, 228, 1-13. https://doi.org/10.1016/j.rse.2019.04.015 (pdf | source code)


In [ ]:
#Installation of geemap preferred as following due to some recent library conflicts

import subprocess

try:
  import geemap
except ImportError:
  print ('geemap package not installed. Installing...')
  subprocess.check_call(["python", '-m', 'pip', 'install', 'geemap'])

#alternatively use

#pip install geemap

In [ ]:
import geemap

Get information about which of the metadata is stored in the GEE S-1 product, the structure of the product, details we want to keep, as well as their index name in GEE.

In [ ]:
geemap.image_props(s1.first()).getInfo()

### Define metadata extractor function to extract the metadata (optional)

In [ ]:
#define function to extract data

def metadata_extractor(image_collection):

  # compute number of images per collection to check the index in ID rows
  collection_count=image_collection.size()
  print('Count: ', str(collection_count.getInfo())+'\n')

  #obtain metadata from image properties for all images in the collection

  image_name =image_collection.aggregate_array('system:index')
  image_name= image_name.getInfo()

  platform_number = image_collection.aggregate_array('platform_number') 
  platform_number= platform_number.getInfo()

  system_time= image_collection.aggregate_array('system:time_start')
  system_time=system_time.getInfo()
  system_time= pd.to_datetime(system_time, unit='ms')

  segment_time =image_collection.aggregate_array('segmentStartTime')
  segment_time=segment_time.getInfo()
  segment_time= pd.to_datetime(segment_time, unit='ms')

  relative_orbit_start=image_collection.aggregate_array('relativeOrbitNumber_start')
  relative_orbit_start=relative_orbit_start.getInfo()

  relative_orbit_stop=image_collection.aggregate_array('relativeOrbitNumber_stop')
  relative_orbit_stop=relative_orbit_stop.getInfo()

  cycle_number = image_collection.aggregate_array('cycleNumber')
  cycle_number = cycle_number.getInfo()

  slice_number = image_collection.aggregate_array('sliceNumber')
  slice_number=slice_number.getInfo()

  coordinates = s1.aggregate_array('system:footprint')
  coordinates = coordinates.getInfo()


  #convert to pandas concatenated dataframe

  image_collection_df = pd.DataFrame({'Image_Name': image_name, 
                                      'Platform_No': platform_number , 
                                      'System_time': system_time, 
                                      'Segment_time': segment_time, 
                                      'RelativeOrbit_start': relative_orbit_start,  
                                      'RelativeOrbit_stop': relative_orbit_stop, 
                                      'Cycle_No': cycle_number, 
                                      'Slice_No': slice_number,
                                      'Coordinates': coordinates})
  
  return image_collection_df

In [ ]:
#extract data and write it in a csv file
s1_meta_csv= metadata_extractor(s1)
s1_meta_csv
s1_meta_csv.to_csv('/content/drive/MyDrive/GEE_files/s1_metadata_test.csv', index=True, header=True)

## Extract Sentinel-1 footprints and get layer extents

In [ ]:
#filter S-1 collection by polarisation (both bands will have the same footprint)
vv_collection = s1.select('VV')

In [ ]:
#extract first product and create new interactive map and visualise product (optional)
vv_image = vv_collection.first()

Map = geemap.Map()
Map.setCenter(lat, lng, 8)
Map.addLayer(vv_image)
Map

In [ ]:
#extract footprints from collection metadata

def footprints_collection (collection):
    footprints_extract = ee.Geometry(collection.get('system:footprint'))
    return ee.Feature(collection).copyProperties(collection, collection.propertyNames())


footprints = ee.FeatureCollection(vv_collection.map(footprints_collection))

In [ ]:
#add collection to map (optional)
Map.addLayer(footprints)

In [ ]:
#download footprints to drive as geoJSON. Alternatively, export as Asset 

export_footprints = ee.batch.Export.table.toDrive(**{
    'collection': footprints,
    'description': 'footprints_for_images',
    'fileFormat': 'GeoJSON',
    'folder':'GEE_files'})

export_footprints.start()

while export_footprints.active():
  print('Polling for task (id: {}).'.format(export_footprints.id))

print('Finished exporting Image footprints')

## Construct json for CDS API requests

JSON Format example:

{
        'product_type': 'reanalysis',
        'format': 'netcdf',
        'variable': [
            '10m_u_component_of_wind', 
            '10m_v_component_of_wind',
        ],
        'year': '2021',
        'month': '10',
        'day': '21',
        'time': [
            '03:00',
        ],
        'area': [
            43.76, 39.87, 40.49,
            41.75,
        ],
    }

To manipulate the geometries in a easier fashion, we will use [geopandas](https://geopandas.org/en/stable/) dataframes.

In [ ]:
!pip install geopandas

In [ ]:
import geopandas as gpd  

Construct geodataframe for the exported geojson

In [ ]:
#read geojson file
footprints_json= gpd.read_file('/content/drive/MyDrive/GEE_files/footprints_for_images.geojson')
footprints_json

In [ ]:
#extract date, time, geometry attributes - the necessary matching attributes. Alternatively, it can be filtered. 

footprints_geom = {'id': footprints_json['id'], 'geometry': footprints_json['geometry'], 'timestamp': footprints_json['system:time_start']}

#Convert date and time from UNIX format
footprints_geom['timestamp'] = pd.to_datetime(footprints_geom['timestamp'], unit='ms')
footprints_geom['date'] = [d.date() for d in footprints_geom['timestamp']]
footprints_geom['time'] = [t.time() for t in footprints_geom['timestamp']]

#construct extents dataframe
extent_df= gpd.GeoDataFrame(footprints_geom)
extent_df

In [ ]:
#extract geometry bounds: x min, x max, y min, y max
bounds= extent_df.bounds
bounds
# extent_df.total_bounds  //returns the total extent of footprints for larger extent of the area

In [ ]:
#merge dataframes to add bounds
bounds_df = gpd.GeoDataFrame(bounds)
key_bounds_df= bounds_df.reset_index()
key_extents_df= extent_df.reset_index()
final_extents_df= key_extents_df.set_index('index').join(key_bounds_df.set_index('index'))
final_extents_df

In [ ]:
#split date and time 
final_extents_df['date'] = pd.to_datetime(final_extents_df['date'], format= '%Y-%m-%d')
final_extents_df['year'] = pd.DatetimeIndex(final_extents_df['date']).year
final_extents_df['month'] = pd.DatetimeIndex(final_extents_df['date']).month
final_extents_df['day']= pd.DatetimeIndex(final_extents_df['date']).day
final_extents_df

In [ ]:
#map new columns - see json request format in CDS API
final_extents_df['area']=final_extents_df.apply(lambda row: [row['maxx'], row['miny'],row['minx'], row['maxy']], axis=1)
final_extents_df['product'] = final_extents_df.apply(lambda row: 'reanalysis', axis=1)
final_extents_df['format'] = final_extents_df.apply(lambda row: 'netcdf', axis=1)
final_extents_df['variables'] = final_extents_df.apply(lambda row: ['10m_u_component_of_wind', '10m_v_component_of_wind'], axis=1)
final_extents_df

In [ ]:
#round acquistion time to the nearest suitable hour to match model output

from datetime import datetime, timedelta, time

def rounder(t):
    if t.minute >= 30 and t.hour !=23:                       #if minutes past 30 and hour is not 23 then round to next hour. e.g: 15:40:25 -> 16:00:00
        return t.replace(second=0, minute=0, hour=t.hour+1)
    elif t.minute >= 30 and t.hour >=23:                     #if minutes past 30 and hour is  23 then round to next hour. e.g: 23:40:25 -> 23:00:00
        return t.replace(second=0, minute=0, hour=t.hour)
    else:
        return t.replace(second=0, minute=0, hour=t.hour)    #if minutes not past 30 and hour is not 23 then round to next hour. e.g: 15:20:25 -> 15:00:00

#create new time filter
for index, row in final_extents_df.iterrows():
  print('Current satellite time is:')
  print (row[4])
  final_extents_df['time_filter'] = final_extents_df.apply(lambda row: rounder(row[4]), axis=1)

print(final_extents_df)

In [ ]:
#get columns index locations for selecting the right index 
names=final_extents_df.columns.values.tolist()
for i in names:
  print (final_extents_df.columns.get_loc(i), i)

In [ ]:
#write json in the right format 

import json
import os

mkdir = '/content/drive/MyDrive/GEE_files/'

def write_json(target_path, target_file, data):
    if not os.path.exists(target_path):
        try:
            os.makedirs(target_path)
        except Exception as e:
            print(e)
            raise
    with open(os.path.join(target_path, target_file), 'w') as f:
        json.dump(data, f)

for index, row in final_extents_df.iterrows():

  json_str =  {'product_type': str(row[13]), 'format': str(row[14]), 'variable': (row[15]), 'year': str(row[9]),'month': str(row[10]), 'day': str(row[11]), 'time':[str(row[16])], 'area': (row[12])}
  json_i= json.dumps(json_str)
  write_json(mkdir, 'download.json', json_i)
 J
  print(json_i)

## Accessing the Copernicus Climate Service API

First will be using the Copernicus Data Store API request....
https://cds.climate.copernicus.eu/api-how-to 

To configure, use the following instructions: https://stackoverflow.com/questions/64304862/using-cdsapi-in-google-colab

In [ ]:
url = 'url: https://cds.climate.copernicus.eu/api/v2'
key = 'key: userkey'

with open('/root/.cdsapirc', 'w') as f:
    f.write('\n'.join([url, key]))

with open('/root/.cdsapirc') as f:
    print(f.read())

In [ ]:
!pip install cdsapi

In [ ]:
import cdsapi
c = cdsapi.Client()

In [ ]:
#make API request and download data
c.retrieve(
    'reanalysis-era5-single-levels',
    json.loads(json_i),
    '/content/drive/MyDrive/GEE_files/download.nc')